# Publish Geoscience objects from a DUF file

This notebook shows how you can login to Evo and publish Geoscience objects from a DUF file to your chosen workspace.

In this first cell we create a ServiceManagerWidget which will open a browser window and ask you to login.

Once logged in, a widget will be displayed below allowing you to select an organisation and workspace to publish objects to.

__Required:__ You must fill in your `client_id` before running this cell.

In [ ]:
from evo.notebooks import ServiceManagerWidget
from evo.oauth import OAuthScopes

manager = await ServiceManagerWidget.with_auth_code(
    client_id="",
    oidc_issuer="https://ims.bentley.com",
    discovery_url="https://discover.api.seequent.com",
    redirect_url="http://localhost:3000/signin-oidc",
    oauth_scopes=OAuthScopes.evo_object
    | OAuthScopes.evo_workspace
    | OAuthScopes.evo_discovery
    | OAuthScopes.offline_access
    | OAuthScopes.openid,
).login()

In the cell below we choose the DUF file we want to publish and set its path in the `duf_file` variable.

Then we choose an EPSG code to use for the Coordinate Reference System.

You may also specify tags to add to the created Geoscience objects.

Then we call `convert_duf`, passing it the DUF file path, EPSG code, the service manager widget from above and finally a
path we want the published objects to appear under.

Then we loop over the results from the `convert_duf` function, printing out each object that was published to Evo.

_Note:_ Some geometry types are not yet supported. A warning will be shown for each element that could not be converted.
 Currently, for publishing we support polyface objects and polylines, and all attribute types except for color
 attributes.

In [ ]:
import os
import pprint

from evo.data_converters.duf.importer import convert_duf

duf_file = os.path.join(os.getcwd(), "data", "input", "Marlin Stopes.duf")
epsg_code = 32650

tags = {"TagName": "Tag value"}
objects_metadata = convert_duf(
    filepath=duf_file,
    epsg_code=epsg_code,
    service_manager_widget=manager,
    tags=tags,
    upload_path="notebook",
    combine_objects_in_layers=True
)

print()
print("These objects have now been published:")

for metadata in objects_metadata:
    pprint.pp(metadata, indent=4)